In [ ]:
import numpy as np
import time

In [ ]:
class node():
    def __init__(self,etat,parent,action,depth,step_cost,path_cost,heuristic_cost):
        self.etat = etat 
        self.parent = parent # node parent
        self.action = action # move up, left, down, right
        self.depth = depth # depth of the node in the tree
        self.step_cost = step_cost # g(n), the cost to take the step
        self.path_cost = path_cost # accumulated g(n), the cost to reach the current node
        self.heuristic_cost = heuristic_cost # h(n), cost to reach goal etat from the current node
        
        # nœud fils
        self.move_up = None 
        self.move_left = None
        self.move_down = None
        self.move_right = None
    
    # see if moving down is valid
    def try_move_down(self):
        # if not possible
        zero_index=[i[0] for i in np.where(self.etat==0)] 
        if zero_index[0] == 0:
            return False
        #if possible
        else:
            up_value = self.etat[zero_index[0]-1,zero_index[1]] # value of the upper carreau
            new_etat = self.etat.copy()
            new_etat[zero_index[0],zero_index[1]] = up_value
            new_etat[zero_index[0]-1,zero_index[1]] = 0
            return new_etat,up_value
        
    # see if moving right is valid
    def try_move_right(self):
      # if not possible
        zero_index=[i[0] for i in np.where(self.etat==0)] 
        if zero_index[1] == 0:
            return False
      # if possible
        else:
            left_value = self.etat[zero_index[0],zero_index[1]-1] # value of the left carreau
            new_etat = self.etat.copy()
            new_etat[zero_index[0],zero_index[1]] = left_value
            new_etat[zero_index[0],zero_index[1]-1] = 0
            return new_etat,left_value
        
    # see if moving up is valid
    def try_move_up(self):
        zero_index=[i[0] for i in np.where(self.etat==0)] 
        if zero_index[0] == 2:
            return False
        else:
            lower_value = self.etat[zero_index[0]+1,zero_index[1]] # value of the lower carreau
            new_etat = self.etat.copy()
            new_etat[zero_index[0],zero_index[1]] = lower_value
            new_etat[zero_index[0]+1,zero_index[1]] = 0
            return new_etat,lower_value
        
    # see if moving left is valid
    def try_move_left(self):
        zero_index=[i[0] for i in np.where(self.etat==0)] 
        if zero_index[1] == 2:
            return False
        else:
            right_value = self.etat[zero_index[0],zero_index[1]+1] # value of the right carreau
            new_etat = self.etat.copy()
            new_etat[zero_index[0],zero_index[1]] = right_value
            new_etat[zero_index[0],zero_index[1]+1] = 0
            return new_etat,right_value
    
    # return user specified heuristic cost
    def get_h_cost(self,new_etat,goal_etat,heuristic_function,path_cost,depth):
        if heuristic_function == 'num_misplaced':
            return self.h_misplaced_cost(new_etat,goal_etat)
    
    # return heuristic cost: number of misplaced carreaus(compare with final node ==> goal)
    def h_misplaced_cost(self,new_etat,goal_etat):
        cost = np.sum(new_etat != goal_etat)-1 # minus 1 to exclude the empty carreau
        if cost > 0:
            return cost
        else:
            return 0 # when all carreaus matches
    
   
        
    # solution path
    def print_path(self):
        # create FILO stacks to place the trace
        etat_trace = [self.etat]
        action_trace = [self.action]
        depth_trace = [self.depth]
        step_cost_trace = [self.step_cost]
        path_cost_trace = [self.path_cost]
        heuristic_cost_trace = [self.heuristic_cost]
        
        # add node information as tracing back up the tree
        while self.parent:
            self = self.parent

            etat_trace.append(self.etat)
            action_trace.append(self.action)
            depth_trace.append(self.depth)
            step_cost_trace.append(self.step_cost)
            path_cost_trace.append(self.path_cost)
            heuristic_cost_trace.append(self.heuristic_cost)

        # print out the path
        step_counter = 0
        while etat_trace:
            print ('step',step_counter)
            print (etat_trace.pop())
            print ('action=',action_trace.pop(),', depth=',str(depth_trace.pop()),\
            ', step cost=',str(step_cost_trace.pop()),', total_cost=',\
            str(path_cost_trace.pop() + heuristic_cost_trace.pop()),'\n')
            
            step_counter += 1
    def depth_first_search(self, goal_etat):
        start = time.time()
        
        pile = [self] # pile of found but unvisited nodes, Pile : first in last out FILO
        pile_num_nodes_popped = 0 # number of nodes popped off the pile, measuring time performance
        pile_max_length = 1 # max number of nodes in the pile, measuring space performance
        
        depth_pile = [0] # pile of node depth
        path_cost_pile = [0] # pile for path cost
        visited = set([]) # record visited etats
        
        while pile:
            # update maximum length of the pile
            if len(pile) > pile_max_length:
                pile_max_length = len(pile)
                
            current_node = pile.pop(0) # select and remove the first node in the pile
            pile_num_nodes_popped += 1 
            
            current_depth = depth_pile.pop(0) # select and remove the depth for current node
            current_path_cost = path_cost_pile.pop(0) # # select and remove the path cost for reaching current node
            visited.add(tuple(current_node.etat.reshape(1,9)[0])) # add etat, which is represented as a tuple
            
            # when the goal etat is found, trace back to the root node and print out the path
            if np.array_equal(current_node.etat,goal_etat):
                current_node.print_path()
                
                print ('Time performance:',str(pile_num_nodes_popped),'nodes popped off the pile.')
                print ('Space performance:', str(pile_max_length),'nodes in the pile at its max.')
                print ('Time spent: %0.2fs' % (time.time()-start))
                return True
            
            else:                
                # see if moving upper carreau down is a valid move
                if current_node.try_move_down():
                    new_etat,up_value = current_node.try_move_down()
                    # check if the resulting node is already visited
                    if tuple(new_etat.reshape(1,9)[0]) not in visited:
                        # create a new child node
                        current_node.move_down = node(etat=new_etat,parent=current_node,action='down',depth=current_depth+1,\
                                              step_cost=up_value,path_cost=current_path_cost+up_value,heuristic_cost=0)
                        pile.insert(0,current_node.move_down)
                        depth_pile.insert(0,current_depth+1)
                        path_cost_pile.insert(0,current_path_cost+up_value)
                    
                # see if moving left carreau to the right is a valid move
                if current_node.try_move_right():
                    new_etat,left_value = current_node.try_move_right()
                    # check if the resulting node is already visited
                    if tuple(new_etat.reshape(1,9)[0]) not in visited:
                        # create a new child node
                        current_node.move_right = node(etat=new_etat,parent=current_node,action='right',depth=current_depth+1,\
                                              step_cost=left_value,path_cost=current_path_cost+left_value,heuristic_cost=0)
                        pile.insert(0,current_node.move_right)
                        depth_pile.insert(0,current_depth+1)
                        path_cost_pile.insert(0,current_path_cost+left_value)
                 
                # see if moving lower carreau up is a valid move
                if current_node.try_move_up():
                    new_etat,lower_value = current_node.try_move_up()
                    # check if the resulting node is already visited
                    if tuple(new_etat.reshape(1,9)[0]) not in visited:
                        # create a new child node
                        current_node.move_up = node(etat=new_etat,parent=current_node,action='up',depth=current_depth+1,\
                                              step_cost=lower_value,path_cost=current_path_cost+lower_value,heuristic_cost=0)
                        pile.insert(0,current_node.move_up)
                        depth_pile.insert(0,current_depth+1)
                        path_cost_pile.insert(0,current_path_cost+lower_value)

                # see if moving right carreau to the left is a valid move
                if current_node.try_move_left():
                    new_etat,right_value = current_node.try_move_left()
                    # check if the resulting node is already visited
                    if tuple(new_etat.reshape(1,9)[0]) not in visited:
                        # create a new child node
                        current_node.move_left = node(etat=new_etat,parent=current_node,action='left',depth=current_depth+1,\
                                              step_cost=right_value,path_cost=current_path_cost+right_value,heuristic_cost=0)
                        pile.insert(0,current_node.move_left)
                        depth_pile.insert(0,current_depth+1)
                        path_cost_pile.insert(0,current_path_cost+right_value)     
    def best_first_search(self, goal_etat):
        start = time.time()
        
        filee= [(self,0)] # filee of (found but unvisited nodes, heuristic cost), ordered by heuristic cost
        filee_num_nodes_popped = 0 # number of nodes popped off the filee, measuring time performance
        filee_max_length = 1 # max number of nodes in the filee, measuring space performance
        
        depth_filee = [(0,0)] # filee of node depth, (depth, heuristic cost)
        path_cost_filee = [(0,0)] # filee for path cost, (path_cost, heuristic cost)
        visited = set([]) # record visited etats
        
        while filee:
            # sort filee based on heuristic cost, in ascending order
            filee = sorted(filee, key=lambda x: x[1])
            depth_filee = sorted(depth_filee, key=lambda x: x[1])
            path_cost_filee= sorted(path_cost_filee, key=lambda x: x[1])
            
            # update maximum length of the filee
            if len(filee) > filee_max_length:
                filee_max_length = len(filee)
                
            current_node = filee.pop(0)[0] # select and remove the first node in the filee
          
            
            filee_num_nodes_popped += 1 
            current_depth = depth_filee.pop(0)[0] # select and remove the depth for current node
            current_path_cost = path_cost_filee.pop(0)[0] # # select and remove the path cost for reaching current node
            visited.add(tuple(current_node.etat.reshape(1,9)[0])) # avoid repeated etat, which is represented as a tuple
            
            # when the goal etat is found, trace back to the root node and print out the path
            if np.array_equal(current_node.etat,goal_etat):
                current_node.print_path()
                
                print ('Time performance:',str(filee_num_nodes_popped),'nodes popped off the filee.')
                print ('Space performance:', str(filee_max_length),'nodes in the filee at its max.')
                print ('Time spent: %0.2fs' % (time.time()-start))
                return True
            
            else:     
                # see if moving upper carreau down is a valid move
                if current_node.try_move_down():
                    new_etat,up_value = current_node.try_move_down()
                    # check if the resulting node is already visited
                    if tuple(new_etat.reshape(1,9)[0]) not in visited:
                        # get heuristic cost
                        h_cost = self.h_misplaced_cost(new_etat,goal_etat)
                        # create a new child node
                        current_node.move_down = node(etat=new_etat,parent=current_node,action='down',depth=current_depth+1,\
                                              step_cost=up_value,path_cost=current_path_cost+up_value,heuristic_cost=h_cost)
                        filee.append((current_node.move_down,h_cost))
                        depth_filee.append((current_depth+1,h_cost))
                        path_cost_filee.append((current_path_cost+up_value,h_cost))
                    
                # see if moving left carreau to the right is a valid move
                if current_node.try_move_right():
                    new_etat,left_value = current_node.try_move_right()
                    # check if the resulting node is already visited
                    if tuple(new_etat.reshape(1,9)[0]) not in visited:
                        # get heuristic cost
                        h_cost = self.h_misplaced_cost(new_etat,goal_etat)
                        # create a new child node
                        current_node.move_right = node(etat=new_etat,parent=current_node,action='right',depth=current_depth+1,\
                                              step_cost=left_value,path_cost=current_path_cost+left_value,heuristic_cost=h_cost)
                        filee.append((current_node.move_right,h_cost))
                        depth_filee.append((current_depth+1,h_cost))
                        path_cost_filee.append((current_path_cost+left_value,h_cost))
                    
                # see if moving lower carreau up is a valid move
                if current_node.try_move_up():
                    new_etat,lower_value = current_node.try_move_up()
                    # check if the resulting node is already visited
                    if tuple(new_etat.reshape(1,9)[0]) not in visited:
                        # get heuristic cost
                        h_cost = self.h_misplaced_cost(new_etat,goal_etat)
                        # create a new child node
                        current_node.move_up = node(etat=new_etat,parent=current_node,action='up',depth=current_depth+1,\
                                              step_cost=lower_value,path_cost=current_path_cost+lower_value,heuristic_cost=h_cost)
                        filee.append((current_node.move_up,h_cost))
                        depth_filee.append((current_depth+1,h_cost))
                        path_cost_filee.append((current_path_cost+lower_value,h_cost))

                # see if moving right carreau to the left is a valid move
                if current_node.try_move_left():
                    new_etat,right_value = current_node.try_move_left()
                    # check if the resulting node is already visited
                    if tuple(new_etat.reshape(1,9)[0]) not in visited:
                        # get heuristic cost
                        h_cost = self.h_misplaced_cost(new_etat,goal_etat)
                        # create a new child node
                        current_node.move_left = node(etat=new_etat,parent=current_node,action='left',depth=current_depth+1,\
                                              step_cost=right_value,path_cost=current_path_cost+right_value,heuristic_cost=h_cost)
                        filee.append((current_node.move_left,h_cost))
                        depth_filee.append((current_depth+1,h_cost))
                        path_cost_filee.append((current_path_cost+right_value,h_cost))
            
    # search based on path cost + heuristic cost
    def a_star_search(self,goal_etat,heuristic_function):
        start = time.time()
        
        queue = [(self,0)] # queue of (found but unvisited nodes, path cost+heuristic cost), ordered by the second element
        queue_num_nodes_popped = 0 # number of nodes popped off the queue, measuring time performance
        queue_max_length = 1 # max number of nodes in the queue, measuring space performance
        
        depth_queue = [(0,0)] # queue of node depth, (depth, path_cost+heuristic cost)
        path_cost_queue = [(0,0)] # queue for path cost, (path_cost, path_cost+heuristic cost)
        visited = set([]) # record visited etats
        
        while queue:
            # sort queue based on path_cost+heuristic cost, in ascending order
            queue = sorted(queue, key=lambda x: x[1])
            depth_queue = sorted(depth_queue, key=lambda x: x[1])
            path_cost_queue = sorted(path_cost_queue, key=lambda x: x[1])
            
            # update maximum length of the queue
            if len(queue) > queue_max_length:
                queue_max_length = len(queue)
                
            current_node = queue.pop(0)[0] # select and remove the first node in the queue
           
            
            queue_num_nodes_popped += 1 
            current_depth = depth_queue.pop(0)[0] # select and remove the depth for current node
            current_path_cost = path_cost_queue.pop(0)[0] # # select and remove the path cost for reaching current node
            visited.add(tuple(current_node.etat.reshape(1,9)[0])) # avoid repeated etat, which is represented as a tuple
            
            # when the goal etat is found, trace back to the root node and print out the path
            if np.array_equal(current_node.etat,goal_etat):
                current_node.print_path()
                
                print ('Time performance:',str(queue_num_nodes_popped),'nodes popped off the queue.')
                print ('Space performance:', str(queue_max_length),'nodes in the queue at its max.')
                print ('Time spent: %0.2fs' % (time.time()-start))
                return True
            
            else:     
                # see if moving upper carreau down is a valid move
                if current_node.try_move_down():
                    new_etat,up_value = current_node.try_move_down()
                    # check if the resulting node is already visited
                    if tuple(new_etat.reshape(1,9)[0]) not in visited:
                        path_cost=current_path_cost+up_value
                        depth = current_depth+1
                        # get heuristic cost
                        h_cost = self.get_h_cost(new_etat,goal_etat,heuristic_function,path_cost,depth)
                        # create a new child node
                        total_cost = path_cost+h_cost
                        current_node.move_down = node(etat=new_etat,parent=current_node,action='down',depth=depth,\
                                              step_cost=up_value,path_cost=path_cost,heuristic_cost=h_cost)
                        queue.append((current_node.move_down, total_cost))
                        depth_queue.append((depth, total_cost))
                        path_cost_queue.append((path_cost, total_cost))
                    
                # see if moving left carreau to the right is a valid move
                if current_node.try_move_right():
                    new_etat,left_value = current_node.try_move_right()
                    # check if the resulting node is already visited
                    if tuple(new_etat.reshape(1,9)[0]) not in visited:
                        path_cost=current_path_cost+left_value
                        depth = current_depth+1
                        # get heuristic cost
                        h_cost = self.get_h_cost(new_etat,goal_etat,heuristic_function,path_cost,depth)
                        # create a new child node
                        #cours:f=g+h
                        total_cost = path_cost+h_cost
                        current_node.move_right = node(etat=new_etat,parent=current_node,action='right',depth=depth,\
                                              step_cost=left_value,path_cost=path_cost,heuristic_cost=h_cost)
                        queue.append((current_node.move_right, total_cost))
                        depth_queue.append((depth, total_cost))
                        path_cost_queue.append((path_cost, total_cost))
                    
                # see if moving lower carreau up is a valid move
                if current_node.try_move_up():
                    new_etat,lower_value = current_node.try_move_up()
                    # check if the resulting node is already visited
                    if tuple(new_etat.reshape(1,9)[0]) not in visited:
                        path_cost=current_path_cost+lower_value
                        depth = current_depth+1
                        # get heuristic cost
                        h_cost = self.get_h_cost(new_etat,goal_etat,heuristic_function,path_cost,depth)
                        # create a new child node
                        total_cost = path_cost+h_cost 
                        current_node.move_up = node(etat=new_etat,parent=current_node,action='up',depth=depth,\
                                              step_cost=lower_value,path_cost=path_cost,heuristic_cost=h_cost)
                        queue.append((current_node.move_up, total_cost))
                        depth_queue.append((depth, total_cost))
                        path_cost_queue.append((path_cost, total_cost))

                # see if moving right carreau to the left is a valid move
                if current_node.try_move_left():
                    new_etat,right_value = current_node.try_move_left()
                    # check if the resulting node is already visited
                    if tuple(new_etat.reshape(1,9)[0]) not in visited:
                        path_cost=current_path_cost+right_value
                        depth = current_depth+1
                        # get heuristic cost
                        h_cost = self.get_h_cost(new_etat,goal_etat,heuristic_function,path_cost,depth)
                        # create a new child node
                        total_cost = path_cost+h_cost
                        current_node.move_left = node(etat=new_etat,parent=current_node,action='left',depth=depth,\
                                              step_cost=right_value,path_cost=path_cost,heuristic_cost=h_cost)
                        queue.append((current_node.move_left, total_cost))
                        depth_queue.append((depth, total_cost))
                        path_cost_queue.append((path_cost, total_cost))


In [ ]:

test = np.array([2,8,3,1,6,4,7,0,5]).reshape(3,3)
initial_etat= test
goal_etat = np.array([1,2,3,8,0,4,7,6,5]).reshape(3,3)
print (initial_etat,'\n')
print (goal_etat)

[[2 8 3]
 [1 6 4]
 [7 0 5]] 

[[1 2 3]
 [8 0 4]
 [7 6 5]]


In [ ]:
root_node = node(etat=initial_etat,parent=None,action=None,depth=0,step_cost=0,path_cost=0,heuristic_cost=0)

In [ ]:
root_node.depth_first_search(goal_etat)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
[[3 2 5]
 [4 0 1]
 [6 8 7]]
action= right , depth= 31541 , step cost= 1 , total_cost= 142810 

step 31542
[[3 2 5]
 [4 8 1]
 [6 0 7]]
action= up , depth= 31542 , step cost= 8 , total_cost= 142818 

step 31543
[[3 2 5]
 [4 8 1]
 [6 7 0]]
action= left , depth= 31543 , step cost= 7 , total_cost= 142825 

step 31544
[[3 2 5]
 [4 8 0]
 [6 7 1]]
action= down , depth= 31544 , step cost= 1 , total_cost= 142826 

step 31545
[[3 2 5]
 [4 0 8]
 [6 7 1]]
action= right , depth= 31545 , step cost= 8 , total_cost= 142834 

step 31546
[[3 2 5]
 [0 4 8]
 [6 7 1]]
action= right , depth= 31546 , step cost= 4 , total_cost= 142838 

step 31547
[[3 2 5]
 [6 4 8]
 [0 7 1]]
action= up , depth= 31547 , step cost= 6 , total_cost= 142844 

step 31548
[[3 2 5]
 [6 4 8]
 [7 0 1]]
action= left , depth= 31548 , step cost= 7 , total_cost= 142851 

step 31549
[[3 2 5]
 [6 4 8]
 [7 1 0]]
action= left , depth= 31549 , step cost= 1 , total_cost

True

In [ ]:
root_node.best_first_search(goal_etat)

step 0
[[2 8 3]
 [1 6 4]
 [7 0 5]]
action= None , depth= 0 , step cost= 0 , total_cost= 0 

step 1
[[2 8 3]
 [1 0 4]
 [7 6 5]]
action= down , depth= 1 , step cost= 6 , total_cost= 8 

step 2
[[2 0 3]
 [1 8 4]
 [7 6 5]]
action= down , depth= 2 , step cost= 8 , total_cost= 17 

step 3
[[0 2 3]
 [1 8 4]
 [7 6 5]]
action= right , depth= 3 , step cost= 2 , total_cost= 18 

step 4
[[1 2 3]
 [0 8 4]
 [7 6 5]]
action= up , depth= 4 , step cost= 1 , total_cost= 18 

step 5
[[1 2 3]
 [8 0 4]
 [7 6 5]]
action= left , depth= 5 , step cost= 8 , total_cost= 25 

Time performance: 6 nodes popped off the filee.
Space performance: 7 nodes in the filee at its max.
Time spent: 0.01s


True

In [ ]:
root_node.a_star_search(goal_etat,heuristic_function = 'num_misplaced')